In [1]:
MARKETS = [
    ('pt', 'br')
]
APP_ID = 'br.com.lojasrenner'

In [2]:
import serpapi

In [ ]:
client = serpapi.Client(api_key='INSERT API KEY HERE')

In [4]:
from datetime import datetime
ingestion_timestamp = datetime.now()

In [5]:
def fetch_reviews_2024(id, hl, gl):
    since = datetime.strptime('2024-01-01', '%Y-%m-%d')
    reviews = []
    pagination_token = None
    try:
        while True:
            print('Fetching reviews...')
            params = {
                'engine': 'google_play_product',
                'product_id': id,
                'store': 'apps',
                'all_reviews': 'true',
                'num': 199,
                'gl': gl,
                'hl': hl,
                'sort_by': 2,
            }
            if pagination_token:
                params['next_page_token'] = pagination_token
            results = client.search(params=params)
            if results.get('error'):
                print(f'Error for {hl}-{gl}: {results['error']}')
                break
            since_reviews = list(
                filter(lambda review: datetime.strptime(review['iso_date'], '%Y-%m-%dT%H:%M:%SZ').date() >= since.date(),
                results['reviews']))
            if len(since_reviews) == 0: break
            reviews.extend(since_reviews)
            pagination_token = results.get('serpapi_pagination', {}).get('next_page_token')
            if not pagination_token: break
        print(f'Reviews fetched for {hl}-{gl}: {len(reviews)}')
        return reviews
    except Exception as e:
        print(f'Error fetching reviews for{hl}-{gl}: {e}')
        return reviews

In [6]:
market_reviews = {}

for market in MARKETS:
    reviews = fetch_reviews_2024(APP_ID, *market)
    market_reviews[f'{market[0]}-{market[1]}'] = reviews

Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Fetching reviews...
Reviews fetched for pt-br: 7737


In [7]:
PROVIDER = 'Renner'
PEER_GROUP = 'FASHION'

import re

import pandas as pd
market_dataframes = {}
ingestion_timestamp = datetime.now()

for market in market_reviews:
    reviews = market_reviews[market]
    if len(reviews) == 0: continue
    lang, country = '', ''
    match = re.match(r'([a-z-]+)-([a-z-]+)', market)
    if match:
        lang, country = match.groups()
    reviews_dict = [
        {
            'app_id': APP_ID,
            'lang': lang,
            'country': country,
            'content': review['snippet'],
            'score': review['rating'],
            'created_at': datetime.strptime(review['iso_date'], '%Y-%m-%dT%H:%M:%SZ'),
            'fetched_at': ingestion_timestamp,
            'provider': PROVIDER,
            'peer_group': PEER_GROUP
        } for review in reviews
    ]
    df = pd.DataFrame(reviews_dict)
    market_dataframes[market] = df

In [8]:
for market in market_dataframes:
    df = market_dataframes[market]
    df.to_csv(f'./{APP_ID}.{market}.csv', index=False)